Emotion Detection

In [ ]:
from pathlib import Path
import cv2
import dlib
import sys
import numpy as np
import argparse
from contextlib import contextmanager
from wide_resnet import WideResNet
from keras.models import load_model
from keras.preprocessing.image import img_to_array

classifier = load_model('emotion_little_vgg_2.h5')
pretrained_model = "https://github.com/yu4u/age-gender-estimation/releases/download/v0.5/weights.28-3.73.hdf5"

In [ ]:
!pip install yt-dlp

In [ ]:
! yt-dlp https://www.youtube.com/watch?v=bk0oury3eeU

[youtube] Extracting URL: https://www.youtube.com/watch?v=bk0oury3eeU
[youtube] bk0oury3eeU: Downloading webpage
[youtube] bk0oury3eeU: Downloading ios player API JSON
[youtube] bk0oury3eeU: Downloading android player API JSON
[youtube] bk0oury3eeU: Downloading player 74a3a562
[youtube] bk0oury3eeU: Downloading m3u8 information
[info] bk0oury3eeU: Downloading 1 format(s): 243+251
[download] Destination: LINC 2.10 - Bad Service in a Restaurant： Lesson 36 - English in Vancouver [bk0oury3eeU].f243.webm
[download] 100% of    6.43MiB in 00:00:00 at 19.76MiB/s
[download] Destination: LINC 2.10 - Bad Service in a Restaurant： Lesson 36 - English in Vancouver [bk0oury3eeU].f251.webm
[download] 100% of    3.16MiB in 00:00:00 at 18.55MiB/s
[Merger] Merging formats into "LINC 2.10 - Bad Service in a Restaurant： Lesson 36 - English in Vancouver [bk0oury3eeU].webm"
Deleting original file LINC 2.10 - Bad Service in a Restaurant： Lesson 36 - English in Vancouver [bk0oury3eeU].f243.webm (pass -k to kee

In [ ]:
! yt-dlp -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' https://www.youtube.com/watch?v=bk0oury3eeU

[youtube] Extracting URL: https://www.youtube.com/watch?v=bk0oury3eeU
[youtube] bk0oury3eeU: Downloading webpage
[youtube] bk0oury3eeU: Downloading ios player API JSON
[youtube] bk0oury3eeU: Downloading android player API JSON
[youtube] bk0oury3eeU: Downloading m3u8 information
[info] bk0oury3eeU: Downloading 1 format(s): 605+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 40
[download] Destination: LINC 2.10 - Bad Service in a Restaurant： Lesson 36 - English in Vancouver [bk0oury3eeU].f605.mp4
[download] 100% of    6.46MiB in 00:00:01 at 3.92MiB/s
[download] Destination: LINC 2.10 - Bad Service in a Restaurant： Lesson 36 - English in Vancouver [bk0oury3eeU].f140.m4a
[download] 100% of    3.06MiB in 00:00:00 at 18.00MiB/s
[Merger] Merging formats into "LINC 2.10 - Bad Service in a Restaurant： Lesson 36 - English in Vancouver [bk0oury3eeU].mp4"
Deleting original file LINC 2.10 - Bad Service in a Restaurant： Lesson 36 - English in Vancouver [bk0oury3eeU].f605.mp4 (

### Detecting Emotion of people in Video

In [ ]:
from os import listdir
from os.path import isfile, join

import cv2
import os

modhash = 'fbe63257a054c1c5466cfd7bf14646d6'
emotion_classes = {0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)


# Define our model parameters
depth = 16
k = 8
weight_file = None
margin = 0.4
image_dir = None


# load model and weights
img_size = 64

detector = dlib.get_frontal_face_detector()

# Initialize Webcam
cap = cv2.VideoCapture('sample2.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video2.mp4', fourcc, 10.0, (int(cap.get(3)), int(cap.get(4))))

count = 0
while True:
    ret, frame = cap.read()

    preprocessed_faces_emo = []

    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(input_img)
    detected = detector(frame, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))

    preprocessed_faces_emo = []
    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
            faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
            face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
            face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
            face_gray_emo = face_gray_emo.astype("float") / 255.0
            face_gray_emo = img_to_array(face_gray_emo)
            face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
            preprocessed_faces_emo.append(face_gray_emo)

        # make a prediction for Emotion
        emo_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces_emo[i])[0]
            emo_labels.append(emotion_classes[preds.argmax()])

        # draw results
        for i, d in enumerate(detected):
            label = "{}".format(emo_labels[i])
            print(label)
            draw_label(frame, (d.left(), d.top()), label)

    out.write(frame)
    count += 1




cap.release()
out.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 38ms/step
Sad
1/1 [==============================] - 0s 41ms/step
Sad
1/1 [==============================] - 0s 35ms/step
Neutral
1/1 [==============================] - 0s 34ms/step
Sad
1/1 [==============================] - 0s 40ms/step
Sad
1/1 [==============================] - 0s 36ms/step
Sad
1/1 [==============================] - 0s 37ms/step
Sad
1/1 [==============================] - 0s 36ms/step
Sad
1/1 [==============================] - 0s 49ms/step
Neutral
1/1 [==============================] - 0s 65ms/step
Neutral
1/1 [==============================] - 0s 33ms/step
Neutral
1/1 [==============================] - 0s 33ms/step
Neutral
1/1 [==============================] - 0s 33ms/step
Neutral
1/1 [==============================] - 0s 36ms/step
Neutral
1/1 [==============================] - 0s 36ms/step
Neutral
1/1 [==============================] - 0s 36ms/step
Neutral
1/1 [==============================] - 0s 34ms/step
Neutral
1/1 [===

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
cap.release()
out.release()